# Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews <br>

EDA: https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/


The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use Score/Rating. A rating of 4 or 5 can be cosnidered as a positive review. A rating of 1 or 2 can be considered as negative one. A review of rating 3 is considered nuetral and such reviews are ignored from our analysis. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




# [1]. Reading Data

## [1.1] Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it is easier to query the data and visualise the data efficiently.
<br> 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score is above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [147]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [148]:
# using SQLite Table to read data.
con = sqlite3.connect('database.sqlite') 

# filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
# SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000, will give top 500000 data points
# you can change the number to any other number based on your computing power

# filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000""", con) 
# for tsne assignment you can take 5k data points

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 50000""", con) 

# Give reviews with Score>3 a positive rating(1), and reviews with a score<3 a negative rating(0).
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (50000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [149]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [150]:
print(display.shape)
display.head()

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [151]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [152]:
display['COUNT(*)'].sum()

393063

#  [2] Exploratory Data Analysis

## [2.1] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [153]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As it can be seen above that same user has multiple reviews with same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [154]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [155]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(46072, 10)

In [156]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

92.144

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [157]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [158]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [159]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(46071, 10)


1    38479
0     7592
Name: Score, dtype: int64

#  [3] Preprocessing

## [3.1].  Preprocessing Review Text

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [160]:
# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

My dogs loves this chicken but its a product from China, so we wont be buying it anymore.  Its very hard to find any chicken products made in the USA but they are out there, but this one isnt.  Its too bad too because its a good product but I wont take any chances till they know what is going on with the china imports.
this is yummy, easy and unusual. it makes a quick, delicous pie, crisp or cobbler. home made is better, but a heck of a lot more work. this is great to have on hand for last minute dessert needs where you really want to impress wih your creativity in cooking! recommended.
Great flavor, low in calories, high in nutrients, high in protein! Usually protein powders are high priced and high in calories, this one is a great bargain and tastes great, I highly recommend for the lady gym rats, probably not "macho" enough for guys since it is soy based...
For those of you wanting a high-quality, yet affordable green tea, you should definitely give this one a try. Let me first star

In [161]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_150 = re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)

print(sent_0)

My dogs loves this chicken but its a product from China, so we wont be buying it anymore.  Its very hard to find any chicken products made in the USA but they are out there, but this one isnt.  Its too bad too because its a good product but I wont take any chances till they know what is going on with the china imports.


In [162]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

My dogs loves this chicken but its a product from China, so we wont be buying it anymore.  Its very hard to find any chicken products made in the USA but they are out there, but this one isnt.  Its too bad too because its a good product but I wont take any chances till they know what is going on with the china imports.
this is yummy, easy and unusual. it makes a quick, delicous pie, crisp or cobbler. home made is better, but a heck of a lot more work. this is great to have on hand for last minute dessert needs where you really want to impress wih your creativity in cooking! recommended.
Great flavor, low in calories, high in nutrients, high in protein! Usually protein powders are high priced and high in calories, this one is a great bargain and tastes great, I highly recommend for the lady gym rats, probably not "macho" enough for guys since it is soy based...
For those of you wanting a high-quality, yet affordable green tea, you should definitely give this one a try. Let me first star

In [163]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [164]:
sent_1500 = decontracted(sent_1500)
print(sent_1500)
print("="*50)

Great flavor, low in calories, high in nutrients, high in protein! Usually protein powders are high priced and high in calories, this one is a great bargain and tastes great, I highly recommend for the lady gym rats, probably not "macho" enough for guys since it is soy based...


In [ ]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

My dogs loves this chicken but its a product from China, so we wont be buying it anymore.  Its very hard to find any chicken products made in the USA but they are out there, but this one isnt.  Its too bad too because its a good product but I wont take any chances till they know what is going on with the china imports.


In [ ]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

Great flavor low in calories high in nutrients high in protein Usually protein powders are high priced and high in calories this one is a great bargain and tastes great I highly recommend for the lady gym rats probably not macho enough for guys since it is soy based 


In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████████████████████████████████████████████████████████████████████| 46071/46071 [00:19<00:00, 2364.66it/s]


In [ ]:
preprocessed_reviews[1500]

'great flavor low calories high nutrients high protein usually protein powders high priced high calories one great bargain tastes great highly recommend lady gym rats probably not macho enough guys since soy based'

<h2><font color='red'>[3.2] Preprocessing Review Summary</font></h2>

In [ ]:
## Similartly you can do preprocessing for review summary also.

In [ ]:
#Importing all relevant libraries

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
# Data Splitting
# Train Data :  60%
# CV Data    :  20%
# Test Data  :  20%

X=preprocessed_reviews
y=np.array(final['Score'])

#####  SPLIT  #######
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.25, random_state=4)


# [4] Featurization

## [4.1] BAG OF WORDS

In [ ]:
#BoW
X_train_bow, X_test_bow, X_cv_bow = X_train, X_test, X_cv
y_train_bow, y_test_bow, y_cv_bow = y_train, y_test, y_cv

count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(X_train_bow)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

bow_counts_train = count_vect.transform(X_train_bow)
bow_counts_cv = count_vect.transform(X_cv_bow)
bow_counts_test = count_vect.transform(X_test_bow)
print("the type of count vectorizer ",type(bow_counts_train))
print("the shape of out text BOW vectorizer ",bow_counts_train.get_shape())
print("the number of unique words ", bow_counts_train.get_shape()[1])
# final_counts_test=final_counts_test.toarray()

some feature names  ['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaaaaaaaaaaa', 'aaaaaaahhhhhh', 'aaaaaawwwwwwwwww', 'aaaaah', 'aaaand', 'aaah']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (27642, 31346)
the number of unique words  31346


## [4.2] Bi-Grams and n-Grams.

In [ ]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
# count_vect = CountVectorizer(ngram_range=(1,2))
# please do read the CountVectorizer documentation http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# you can choose these numebrs min_df=10, max_features=5000, of your choice
count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
final_bigram_counts = count_vect.fit_transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (46071, 5000)
the number of unique words including both unigrams and bigrams  5000


## [4.3] TF-IDF

In [ ]:
X_train_tf, X_test_tf, X_cv_tf = X_train, X_test, X_cv
y_train_tf, y_test_tf, y_cv_tf = y_train, y_test, y_cv

tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
final_tf_idf_train = tf_idf_vect.fit_transform(X_train_tf)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)


final_tf_idf_cv = tf_idf_vect.transform(X_cv_tf)
final_tf_idf_test = tf_idf_vect.transform(X_test_tf)
print("the type of count vectorizer ",type(final_tf_idf_train))
print("the shape of out text TFIDF vectorizer ",final_tf_idf_train.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf_train.get_shape()[1])

some sample features(unique words in the corpus) ['ability', 'able', 'able buy', 'able drink', 'able eat', 'able enjoy', 'able find', 'able get', 'able make', 'able order']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (27642, 16605)
the number of unique words including both unigrams and bigrams  16605


## [4.4] Word2Vec

In [ ]:
# Train your own Word2Vec model using your own text corpus

X_train_w2v, X_test_w2v, X_cv_w2v = X_train, X_test, X_cv
y_train_w2v, y_test_w2v, y_cv_w2v = y_train, y_test, y_cv


i=0
list_of_sentance_test=[]
for sentance in X_test_w2v:
    list_of_sentance_test.append(sentance.split())

i=0
list_of_sentance_cv=[]
for sentance in X_cv_w2v:
    list_of_sentance_cv.append(sentance.split())

i=0
list_of_sentance=[]
for sentance in X_train_w2v:
    list_of_sentance.append(sentance.split())

In [ ]:
# Using Google News Word2Vectors

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W17SRFAzZPY
# you can comment this whole cell
# or change these varible according to your need

is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")
# print("\n"+"#"*60)
# print(len(X_train_w2v),len(y_train_w2v))
# print(len(X_cv_w2v),len(y_cv_w2v))
# print(len(X_test_w2v),len(y_test_w2v))

[('awesome', 0.8272461295127869), ('fantastic', 0.7835323810577393), ('wonderful', 0.7762249708175659), ('good', 0.7726205587387085), ('excellent', 0.771723747253418), ('terrific', 0.7572211027145386), ('amazing', 0.7447072267532349), ('perfect', 0.7228223085403442), ('decent', 0.6722276210784912), ('outstanding', 0.6442440152168274)]
[('best', 0.7512112259864807), ('experienced', 0.743771493434906), ('tastiest', 0.7263764142990112), ('closest', 0.6903558969497681), ('smoothest', 0.6801620721817017), ('hooked', 0.6770093441009521), ('ever', 0.6718454360961914), ('horrible', 0.6652827858924866), ('exceeds', 0.665189802646637), ('eaten', 0.6647264957427979)]


In [ ]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  10047
sample words  ['thought', 'wonderful', 'purchased', 'share', 'family', 'hit', 'dad', 'kept', 'sneaking', 'dinner', 'eventually', 'entire', 'tin', 'disappeared', 'seen', 'every', 'awhile', 'mood', 'one', 'fudge', 'comes', 'bag', 'inside', 'piece', 'individually', 'wrapped', 'moment', 'opened', 'packaged', 'wondered', 'would', 'taste', 'strongly', 'alcohol', 'smelled', 'not', 'something', 'like', 'rather', 'surprised', 'hint', 'sweet', 'quite', 'enjoyable', 'highly', 'recommend', 'puppies', 'super', 'small', 'last']


## [4.4.1] Converting text into vectors using Avg W2V, TFIDF-W2V

#### [4.4.1.1] Avg W2v

In [ ]:
# TRAINING W2V DATA VECTORS

# average Word2Vec
# compute average word2vec for each review.
sent_vectors_train = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_train.append(sent_vec)
print(len(sent_vectors_train))
print(len(sent_vectors_train[0]))

100%|███████████████████████████████████████████████████████████████████████████| 27642/27642 [00:35<00:00, 781.78it/s]


27642
50


In [ ]:
# CV W2V DATA VECTORS

sent_vectors_cv = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_cv): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_cv.append(sent_vec)
print(len(sent_vectors_cv), len(y_cv_w2v))
print(len(sent_vectors_cv[0]))

100%|█████████████████████████████████████████████████████████████████████████████| 9214/9214 [00:11<00:00, 785.55it/s]


9214 9214
50


In [ ]:
# TEST W2V DATA VECTORS

sent_vectors_test = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_test): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_test.append(sent_vec)
print(len(sent_vectors_test))
print(len(sent_vectors_test[0]))

100%|█████████████████████████████████████████████████████████████████████████████| 9215/9215 [00:12<00:00, 742.21it/s]


9215
50


#### [4.4.1.2] TFIDF weighted W2v

In [ ]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
tf_idf_matrix = model.fit_transform(X_train_w2v)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [ ]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors_train = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors_train.append(sent_vec)
    row += 1

 28%|█████████████████████▊                                                       | 7849/27642 [02:11<04:42, 70.16it/s]

In [ ]:

tfidf_sent_vectors_cv = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance_cv): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors_cv.append(sent_vec)
    row += 1

In [ ]:

tfidf_sent_vectors_test = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance_test): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors_test.append(sent_vec)
    row += 1

# [5] Assignment 3: KNN

<ol>
    <li><strong>Apply Knn(brute force version) on these feature sets</strong>
        <ul>
            <li><font color='red'>SET 1:</font>Review text, preprocessed one converted into vectors using (BOW)</li>
            <li><font color='red'>SET 2:</font>Review text, preprocessed one converted into vectors using (TFIDF)</li>
            <li><font color='red'>SET 3:</font>Review text, preprocessed one converted into vectors using (AVG W2v)</li>
            <li><font color='red'>SET 4:</font>Review text, preprocessed one converted into vectors using (TFIDF W2v)</li>
        </ul>
    </li>
    <br>
    <li><strong>Apply Knn(kd tree version) on these feature sets</strong>
        <br><font color='red'>NOTE: </font>sklearn implementation of kd-tree accepts only dense matrices, you need to convert the sparse matrices of CountVectorizer/TfidfVectorizer into dense matices. You can convert sparse matrices to dense using .toarray() attribute. For more information please visit this <a href='https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.toarray.html'>link</a>
        <ul>
            <li><font color='red'>SET 5:</font>Review text, preprocessed one converted into vectors using (BOW) but with restriction on maximum features generated.
            <pre>
            count_vect = CountVectorizer(min_df=10, max_features=500) 
            count_vect.fit(preprocessed_reviews)
            </pre>
            </li>
            <li><font color='red'>SET 6:</font>Review text, preprocessed one converted into vectors using (TFIDF) but with restriction on maximum features generated.
            <pre>
                tf_idf_vect = TfidfVectorizer(min_df=10, max_features=500)
                tf_idf_vect.fit(preprocessed_reviews)
            </pre>
            </li>
            <li><font color='red'>SET 3:</font>Review text, preprocessed one converted into vectors using (AVG W2v)</li>
            <li><font color='red'>SET 4:</font>Review text, preprocessed one converted into vectors using (TFIDF W2v)</li>
        </ul>
    </li>
    <br>
    <li><strong>The hyper paramter tuning(find best K)</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>Find the best hyper paramter using k-fold cross validation or simple cross validation data</li>
    <li>Use gridsearch cv or randomsearch cv or you can also write your own for loops to do this task of hyperparameter tuning</li>
        </ul>
    </li>
    <br>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='train_cv_auc.JPG' width=300px></li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='train_test_auc.JPG' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='confusion_matrix.png' width=300px></li>
        </ul>
    </li>
    <br>
    <li><strong>Conclusion</strong>
        <ul>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format. To print out a table please refer to this prettytable library<a href='http://zetcode.com/python/prettytable/'> link</a> 
        <img src='summary.JPG' width=400px>
    </li>
        </ul>
</ol>

<h4><font color='red'>Note: Data Leakage</font></h4>

1. There will be an issue of data-leakage if you vectorize the entire data and then split it into train/cv/test.
2. To avoid the issue of data-leakag, make sure to split your data first and then vectorize it. 
3. While vectorizing your data, apply the method fit_transform() on you train data, and apply the method transform() on cv/test data.
4. For more details please go through this <a href='https://soundcloud.com/applied-ai-course/leakage-bow-and-tfidf'>link.</a>

## [5.1] Applying KNN brute force

### [5.1.1] Applying KNN brute force on BOW,<font color='red'> SET 1</font>

In [ ]:
# Please write all the code with proper documentation
X_train_bow, X_test_bow, y_train, y_test, X_cv_bow, y_cv = bow_counts_train, bow_counts_test, y_train, y_test, bow_counts_cv, y_cv


auc_train=[]
ks_train=[]
auc_cv=[]
ks_cv=[]

############ HYPER-PARAMETER TUNING ##############
for i in range(1,50,4):
    knn = KNeighborsClassifier(n_neighbors=i, algorithm='brute')
    knn.fit(X_train_bow, y_train)
    # y_pred = knn.predict(X_test_bow)
    proba = knn.predict_proba(X_train_bow)[:,1]
    auc_train.append(roc_auc_score(y_train, proba))
    ks_train.append(i)

    proba = knn.predict_proba(X_cv_bow)[:,1]
    auc_cv.append(roc_auc_score(y_cv, proba))
    ks_cv.append(i)

# print("Pre-def accuracy: ", metrics.accuracy_score(y_test, y_pred))


In [ ]:
######## KNN BRUTE-FORCE AUC PLOTTING ########
ax=sns.lineplot(ks_train,auc_train)
ax=sns.lineplot(ks_cv,auc_cv)
ax.set(xlabel='Differnt K values',ylabel='roc_auc_score')
ax.set_title('AUC Score vs Different K value Curve')
plt.show()

###############################################

############## BEST HYPER-PARAMETER ###########
best_k = ks_cv[auc_cv.index(max(auc_cv))]
print("Best K for KNN Model from cv_data is: ",best_k, "for cooresponding AUC value of:",max(auc_cv))


In [ ]:

######### FRESH-MODEL FOR BEST_K ######################
knn = KNeighborsClassifier(n_neighbors=best_k,algorithm='brute')
knn.fit(X_train_bow, y_train)

y_pred = knn.predict(X_test_bow)
proba = knn.predict_proba(X_test_bow)[:,1]
fpr, tpr, threshold = roc_curve(y_test, proba)
roc_auc = auc(fpr, tpr)
print("AUC score on test_data with best_k: ",roc_auc)
proba = knn.predict_proba(X_train_bow)[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(y_train, proba)
roc_auc_train = auc(fpr_train, tpr_train)

k_1 = best_k
set_1_auc = roc_auc
############# ROC CURVE ######################
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'g', label = 'AUC_train = %0.3f' % roc_auc_train)
plt.plot(fpr, tpr, 'b', label = 'AUC_test = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()

In [ ]:
########### CONFUSION MATRIX ############

y_true = y_test
y_pred = y_pred

array = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(array, index = [i for i in "01"],columns = [i for i in "01"])

plt.figure(figsize = (5,5))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, linewidths=6.3, linecolor='pink', cbar=False, square=True)
plt.show()

### [5.1.2] Applying KNN brute force on TFIDF,<font color='red'> SET 2</font>

In [ ]:
# Please write all the code with proper documentation

X_train_tf, X_test_tf, X_cv_tf = final_tf_idf_train, final_tf_idf_test,final_tf_idf_cv

auc_train=[]
ks_train=[]
auc_cv=[]
ks_cv=[]

for i in range(1,50,6):
    knn = KNeighborsClassifier(n_neighbors=i, algorithm='brute', metric='cosine')
    knn.fit(X_train_tf, y_train_tf)
    # y_pred = knn.predict(X_test)
    proba = knn.predict_proba(X_train_tf)[:,1]
    auc_train.append(roc_auc_score(y_train_tf, proba))
    ks_train.append(i)
    
    proba = knn.predict_proba(X_cv_tf)[:,1]
    auc_cv.append(roc_auc_score(y_cv_tf, proba))
    ks_cv.append(i)

# print("Pre-def accuracy: ", metrics.accuracy_score(y_test, y_pred))


In [ ]:

######## AUC PLOTTING ########
ax=sns.lineplot(ks_train,auc_train,dashes=True)
ax=sns.lineplot(ks_cv,auc_cv)
ax.set(xlabel='Differnt K values',ylabel='roc_auc_score')
ax.set_title('AUC Score vs Different K value Curve')
plt.show()
######## AUC PLOTTING ########


######## best_k ##############
best_k = ks_cv[auc_cv.index(max(auc_cv))]
print("Best K for KNN Model from cv_data is: ",best_k, "for cooresponding AUC value of:",max(auc_cv))

In [ ]:

######### FRESH-MODEL FOR BEST_K ######################
knn = KNeighborsClassifier(n_neighbors=best_k,algorithm='brute', metric='cosine')
knn.fit(X_train_tf, y_train_tf)
y_pred = knn.predict(X_test_tf)


proba = knn.predict_proba(X_test_tf)[:,1]
fpr, tpr, threshold = roc_curve(y_test_tf, proba)
roc_auc = auc(fpr, tpr)
print("AUC score on test_data with best_k: ",roc_auc)
#############################################
proba = knn.predict_proba(X_train_tf)[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(y_train_tf, proba)
roc_auc_train = auc(fpr_train, tpr_train)

k_2 = best_k
set_2_auc = roc_auc
############# ROC ######################
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'g', label = 'AUC_train = %0.2f' % roc_auc_train)
plt.plot(fpr, tpr, 'b', label = 'AUC_test = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()

In [ ]:
y_true = y_test_tf
y_pred = y_pred

############ CONFUSION MATRIX ##################
array = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(array, index = [i for i in "01"],
                  columns = [i for i in "01"])

plt.figure(figsize = (10,7))
plt.title('Heatmap Confusion Matrix')
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, linewidths=8.3, linecolor='pink', cbar=False, square=True)
plt.show()

# y_test_tf
# unique, counts = np.unique(y_test_tf, return_counts=True)
# dict(zip(unique, counts))

### [5.1.3] Applying KNN brute force on AVG W2V,<font color='red'> SET 3</font>

In [ ]:
# Please write all the code with proper documentation

X_train_avgw2v, X_cv_avgw2v, X_test_avgw2v = sent_vectors_train, sent_vectors_cv, sent_vectors_test
auc_train=[]
ks_train=[]
auc_cv=[]
ks_cv=[]
for i in range(1,50,4):
    knn = KNeighborsClassifier(n_neighbors=i, algorithm='brute') #try distance cosine too
    knn.fit(X_train_avgw2v, y_train_w2v)
    # y_pred = knn.predict(X_test_avgw2v)
    proba = knn.predict_proba(X_train_avgw2v)[:,1]
    auc_train.append(roc_auc_score(y_train_w2v, proba))
    ks_train.append(i)
    
    proba = knn.predict_proba(X_cv_avgw2v)[:,1]
    auc_cv.append(roc_auc_score(y_cv_w2v, proba))
    ks_cv.append(i)

In [ ]:

######## AUC PLOTTING ########
ax=sns.lineplot(ks_train,auc_train)
ax=sns.lineplot(ks_cv,auc_cv)
ax.set(xlabel='Differnt K values',ylabel='roc_auc_score')
ax.set_title('AUC Score vs Different K value Curve')
plt.show()

######## best_k ##############
best_k = ks_cv[auc_cv.index(max(auc_cv))]
print("Best K for KNN Model from cv_data is: ",best_k, "for cooresponding AUC value of:",max(auc_cv))


In [ ]:

######### FRESH-MODEL FOR BEST_K ######################
knn = KNeighborsClassifier(n_neighbors=best_k,algorithm='brute')
knn.fit(X_train_avgw2v, y_train_w2v)
y_pred = knn.predict(X_test_avgw2v)
proba = knn.predict_proba(X_test_avgw2v)[:,1]


fpr, tpr, threshold = roc_curve(y_test_w2v, proba)
roc_auc = auc(fpr, tpr)
print("AUC score on test_data with best_k: ",roc_auc)
#############################################
proba = knn.predict_proba(X_train_avgw2v)[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(y_train_w2v, proba)
roc_auc_train = auc(fpr_train, tpr_train)

k_3 = best_k
set_3_auc = roc_auc
############# ROC ######################
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'g', label = 'AUC_train = %0.3f' % roc_auc_train)
plt.plot(fpr, tpr, 'b', label = 'AUC_test = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()


In [ ]:
########## CONFUSION MATRIX ###########

y_true = y_test_w2v
y_pred = y_pred

array = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(array, index = [i for i in "01"],
                  columns = [i for i in "01"])


plt.figure(figsize = (5,5))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, linewidths=6.3, linecolor='pink', cbar=False, square=True)
plt.show()

### [5.1.4] Applying KNN brute force on TFIDF W2V,<font color='red'> SET 4</font>

In [ ]:
# Please write all the code with proper documentation

X_train_tfw2v, X_cv_w2v, X_test_tfw2v = tfidf_sent_vectors_train, tfidf_sent_vectors_cv, tfidf_sent_vectors_test

auc_train=[]
ks_train=[]
auc_cv=[]
ks_cv=[]

for i in range(1,50,4):
    knn = KNeighborsClassifier(n_neighbors=i, algorithm='brute') #try distance cosine too
    knn.fit(X_train_tfw2v, y_train_w2v)
    # y_pred = knn.predict(X_test_tfw2v)
    proba = knn.predict_proba(X_train_tfw2v)[:,1]
    auc_train.append(roc_auc_score(y_train_w2v, proba))
    ks_train.append(i)
    #############################################################
    proba = knn.predict_proba(X_cv_w2v)[:,1]
    auc_cv.append(roc_auc_score(y_cv_w2v, proba))
    ks_cv.append(i)


In [ ]:
######## AUC PLOTTING ########
ax=sns.lineplot(ks_train,auc_train)
ax=sns.lineplot(ks_cv,auc_cv)
ax.set(xlabel='Differnt K values',ylabel='roc_auc_score')
ax.set_title('AUC Score vs Different K value Curve')
plt.show()

######## best_k ##############A
best_k = ks_cv[auc_cv.index(max(auc_cv))]
print("Best K for KNN Model from cv_data is: ",best_k, "for cooresponding AUC value of:",max(auc_cv))


In [ ]:


######### FRESH-MODEL FOR BEST_K ######################
knn = KNeighborsClassifier(n_neighbors=best_k,algorithm='brute')
knn.fit(X_train_tfw2v, y_train_w2v)
y_pred = knn.predict(X_test_tfw2v)
proba = knn.predict_proba(X_test_tfw2v)[:,1]


fpr, tpr, threshold = roc_curve(y_test_w2v, proba)
roc_auc = auc(fpr, tpr)
print("AUC score on test_data with best_k: ",roc_auc)
#############################################
proba = knn.predict_proba(X_train_tfw2v)[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(y_train_w2v, proba)
roc_auc_train = auc(fpr_train, tpr_train)

k_4 = best_k
set_4_auc = roc_auc
############# ROC ######################
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'g', label = 'AUC_train = %0.3f' % roc_auc_train)
plt.plot(fpr, tpr, 'b', label = 'AUC_test = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()


In [ ]:


y_true = y_test_w2v
y_pred = y_pred

from sklearn.metrics import confusion_matrix

array = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(array, index = [i for i in "01"],
                  columns = [i for i in "01"])


plt.figure(figsize = (5,5))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, linewidths=6.3, linecolor='pink', cbar=False, square=True)
plt.show()

## [5.2] Applying KNN kd-tree

### [5.2.1] Applying KNN kd-tree on BOW,<font color='red'> SET 5</font>

In [ ]:
# Please write all the code with proper documentation

# Using only 20k data points:
X_temp= X[:20000]
y_temp= y[:20000]

#####  SPLIT  #######
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_temp, y_temp, test_size=0.2, random_state=4)
X_train_bow, X_cv_bow, y_train_bow, y_cv_bow = train_test_split(X_train_bow, y_train_bow, test_size=0.25, random_state=4)

##### BOW Vectorization ######
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(X_train_bow)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

bow_counts_train = count_vect.transform(X_train_bow)
bow_counts_cv = count_vect.transform(X_cv_bow)
bow_counts_test = count_vect.transform(X_test_bow)
print("the type of count vectorizer ",type(bow_counts_train))
print("the shape of out text BOW vectorizer ",bow_counts_train.get_shape())
print("the number of unique words ", bow_counts_train.get_shape()[1])
bow_counts_test=bow_counts_test.toarray()

In [ ]:
# Please write all the code with proper documentation
X_train_bow, X_test_bow, y_train, y_test, X_cv_bow, y_cv = bow_counts_train, bow_counts_test, y_train_bow, y_test_bow, bow_counts_cv, y_cv_bow


auc_train=[]
ks_train=[]
auc_cv=[]
ks_cv=[]

############ HYPER-PARAMETER TUNING ##############
for i in range(1,50,4):
    knn = KNeighborsClassifier(n_neighbors=i, algorithm='kd_tree')
    knn.fit(X_train_bow, y_train)
    # y_pred = knn.predict(X_test_bow)
    proba = knn.predict_proba(X_train_bow)[:,1]
    auc_train.append(roc_auc_score(y_train, proba))
    ks_train.append(i)

    proba = knn.predict_proba(X_cv_bow)[:,1]
    auc_cv.append(roc_auc_score(y_cv, proba))
    ks_cv.append(i)

# print("Pre-def accuracy: ", metrics.accuracy_score(y_test, y_pred))


In [ ]:
######## KNN BRUTE-FORCE AUC PLOTTING ########
ax=sns.lineplot(ks_train,auc_train)
ax=sns.lineplot(ks_cv,auc_cv)
ax.set(xlabel='Differnt K values',ylabel='roc_auc_score')
ax.set_title('AUC Score vs Different K value Curve')
plt.show()

############## BEST HYPER-PARAMETER ###########
best_k = ks_cv[auc_cv.index(max(auc_cv))]
print("Best K for KNN Model from cv_data is: ",best_k, "for cooresponding AUC value of:",max(auc_cv))


In [ ]:

######### FRESH-MODEL FOR BEST_K ######################
knn = KNeighborsClassifier(n_neighbors=best_k,algorithm='kd_tree')
knn.fit(X_train_bow, y_train)

y_pred = knn.predict(X_test_bow)
proba = knn.predict_proba(X_test_bow)[:,1]
fpr, tpr, threshold = roc_curve(y_test, proba)
roc_auc = auc(fpr, tpr)
print("AUC score on test_data with best_k: ",roc_auc)

proba = knn.predict_proba(X_train_bow)[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(y_train, proba)
roc_auc_train = auc(fpr_train, tpr_train)

k_5 = best_k
set_5_auc = roc_auc
############# ROC CURVE ######################
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'g', label = 'AUC_train = %0.3f' % roc_auc_train)
plt.plot(fpr, tpr, 'b', label = 'AUC_test = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()

In [ ]:
########### CONFUSION MATRIX ############

y_true = y_test
y_pred = y_pred

array = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(array, index = [i for i in "01"],columns = [i for i in "01"])

plt.figure(figsize = (5,5))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, linewidths=6.3, linecolor='pink', cbar=False, square=True)
plt.show()

### [5.2.2] Applying KNN kd-tree on TFIDF,<font color='red'> SET 6</font>

In [ ]:
# Please write all the code with proper documentation

# Split
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_temp, y_temp, test_size=0.3, random_state=4)
X_train_tf, X_cv_tf, y_train_tf, y_cv_tf = train_test_split(X_train_tf, y_train_tf, test_size=0.3, random_state=4)
# print(len(X_train_tf),len(X_test_tf),len(y_train_tf),len(y_test_tf),len(y_cv_tf),len(X_cv_tf))

# TFIDF Vectorisation
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
final_tf_idf_train = tf_idf_vect.fit_transform(X_train_tf)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)


final_tf_idf_cv = tf_idf_vect.transform(X_cv_tf)
final_tf_idf_test = tf_idf_vect.transform(X_test_tf)
print("the type of count vectorizer ",type(final_tf_idf_train))
print("the shape of out text TFIDF vectorizer ",final_tf_idf_train.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf_train.get_shape()[1])

final_tf_idf_train =final_tf_idf_train.toarray()
final_tf_idf_test =final_tf_idf_test.toarray()
final_tf_idf_cv =final_tf_idf_cv.toarray()

In [ ]:
# Please write all the code with proper documentation

X_train_tf, X_test_tf, X_cv_tf = final_tf_idf_train, final_tf_idf_test,final_tf_idf_cv

auc_train=[]
ks_train=[]
auc_cv=[]
ks_cv=[]

for i in range(1,50,6):
    knn = KNeighborsClassifier(n_neighbors=i, algorithm='kd_tree')
    knn.fit(X_train_tf, y_train_tf)
    # y_pred = knn.predict(X_test)
    proba = knn.predict_proba(X_train_tf)[:,1]
    auc_train.append(roc_auc_score(y_train_tf, proba))
    ks_train.append(i)
    
    proba = knn.predict_proba(X_cv_tf)[:,1]
    auc_cv.append(roc_auc_score(y_cv_tf, proba))
    ks_cv.append(i)

# print("Pre-def accuracy: ", metrics.accuracy_score(y_test, y_pred))


In [ ]:

######## AUC PLOTTING ########
ax=sns.lineplot(ks_train,auc_train,dashes=True)
ax=sns.lineplot(ks_cv,auc_cv)
ax.set(xlabel='Differnt K values',ylabel='roc_auc_score')
ax.set_title('AUC Score vs Different K value Curve')
plt.show()
######## AUC PLOTTING ########


######## best_k ##############
best_k = ks_cv[auc_cv.index(max(auc_cv))]
print("Best K for KNN Model from cv_data is: ",best_k, "for cooresponding AUC value of:",max(auc_cv))

In [ ]:

######### FRESH-MODEL FOR BEST_K ######################
knn = KNeighborsClassifier(n_neighbors=best_k,algorithm='brute', metric='cosine')
knn.fit(X_train_tf, y_train_tf)
y_pred = knn.predict(X_test_tf)


proba = knn.predict_proba(X_test_tf)[:,1]
fpr, tpr, threshold = roc_curve(y_test_tf, proba)
roc_auc = auc(fpr, tpr)
print("AUC score on test_data with best_k: ",roc_auc)
#############################################
proba = knn.predict_proba(X_train_tf)[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(y_train_tf, proba)
roc_auc_train = auc(fpr_train, tpr_train)

k_6 = best_k
set_6_auc = roc_auc
############# ROC ######################
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'g', label = 'AUC_train = %0.2f' % roc_auc_train)
plt.plot(fpr, tpr, 'b', label = 'AUC_test = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()

In [ ]:
y_true = y_test_tf
y_pred = y_pred

############ CONFUSION MATRIX ##################
array = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(array, index = [i for i in "01"],
                  columns = [i for i in "01"])

plt.title('Heatmap Confusion Matrix')
plt.figure(figsize = (5,5))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, linewidths=6.3, linecolor='pink', cbar=False, square=True)
plt.show()
# y_test_tf
# unique, counts = np.unique(y_test_tf, return_counts=True)
# dict(zip(unique, counts))

### [5.2.3] Applying KNN kd-tree on AVG W2V,<font color='red'> SET 3</font>

### [5.2.3.1] Word2Vec for 20k data-points


In [ ]:
# Train your own Word2Vec model using your own text corpus


#####  SPLIT  #######
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_temp, y_temp, test_size=0.2, random_state=4)
X_train_w2v, X_cv_w2v, y_train_w2v, y_cv_w2v = train_test_split(X_train_w2v, y_train_w2v, test_size=0.25, random_state=4)

i=0
list_of_sentance_test=[]
for sentance in X_test_w2v:
    list_of_sentance_test.append(sentance.split())

i=0
list_of_sentance_cv=[]
for sentance in X_cv_w2v:
    list_of_sentance_cv.append(sentance.split())

i=0
list_of_sentance=[]
for sentance in X_train_w2v:
    list_of_sentance.append(sentance.split())

In [ ]:
# Using Google News Word2Vectors

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W17SRFAzZPY
# you can comment this whole cell
# or change these varible according to your need

is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")
# print("\n"+"#"*60)
# print(len(X_train_w2v),len(y_train_w2v))
# print(len(X_cv_w2v),len(y_cv_w2v))
# print(len(X_test_w2v),len(y_test_w2v))

In [ ]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

### [5.2.3.2] Converting text into vectors using Avg W2V, TFIDF-W2V for 20k data-points

####  Avg W2v


In [ ]:


# TRAINING W2V DATA VECTORS
# average Word2Vec
# compute average word2vec for each review.
sent_vectors_train = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_train.append(sent_vec)
print(len(sent_vectors_train))
print(len(sent_vectors_train[0]))

In [ ]:
# CV W2V DATA VECTORS

sent_vectors_cv = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_cv): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_cv.append(sent_vec)
print(len(sent_vectors_cv), len(y_cv_w2v))
print(len(sent_vectors_cv[0]))

In [ ]:
# TEST W2V DATA VECTORS

sent_vectors_test = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_test): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_test.append(sent_vec)
print(len(sent_vectors_test))
print(len(sent_vectors_test[0]))

In [ ]:
# Please write all the code with proper documentation

auc_train=[]
ks_train=[]
auc_cv=[]
ks_cv=[]
for i in tqdm(range(1,50,6)):
    knn = KNeighborsClassifier(n_neighbors=i, algorithm='kd_tree') #try distance cosine too
    knn.fit(X_train_avgw2v, y_train_w2v)
    # y_pred = knn.predict(X_test_avgw2v)
    proba = knn.predict_proba(X_train_avgw2v)[:,1]
    auc_train.append(roc_auc_score(y_train_w2v, proba))
    ks_train.append(i)
    #############################################################
    proba = knn.predict_proba(X_cv_avgw2v)[:,1]
    auc_cv.append(roc_auc_score(y_cv_w2v, proba))
    ks_cv.append(i)

In [ ]:

######## AUC PLOTTING ########
ax=sns.lineplot(ks_train,auc_train)
ax=sns.lineplot(ks_cv,auc_cv)
ax.set(xlabel='Differnt K values',ylabel='roc_auc_score')
ax.set_title('AUC Score vs Different K value Curve')
plt.show()

######## best_k ##############
best_k = ks_cv[auc_cv.index(max(auc_cv))]
print("Best K for KNN Model from cv_data is: ",best_k, "for cooresponding AUC value of:",max(auc_cv))


In [ ]:

######### FRESH-MODEL FOR BEST_K ######################
knn = KNeighborsClassifier(n_neighbors=best_k,algorithm='brute')
knn.fit(X_train_avgw2v, y_train_w2v)
y_pred = knn.predict(X_test_avgw2v)
proba = knn.predict_proba(X_test_avgw2v)[:,1]


fpr, tpr, threshold = roc_curve(y_test_w2v, proba)
roc_auc = auc(fpr, tpr)
print("AUC score on test_data with best_k: ",roc_auc)
#############################################
proba = knn.predict_proba(X_train_avgw2v)[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(y_train_w2v, proba)
roc_auc_train = auc(fpr_train, tpr_train)

k_7 = best_k
set_7_auc = roc_auc
############# ROC ######################
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'g', label = 'AUC_train = %0.3f' % roc_auc_train)
plt.plot(fpr, tpr, 'b', label = 'AUC_test = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()


In [ ]:
########## CONFUSION MATRIX ###########

y_true = y_test_w2v
y_pred = y_pred

array = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(array, index = [i for i in "01"],
                  columns = [i for i in "01"])


plt.figure(figsize = (5,5))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, linewidths=6.3, linecolor='pink', cbar=False, square=True)
plt.show()

### [5.2.4] Applying KNN kd-tree on TFIDF W2V,<font color='red'> SET 4</font>

####  TFIDF weighted W2v

In [ ]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
tf_idf_matrix = model.fit_transform(X_train_w2v)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [ ]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors_train = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors_train.append(sent_vec)
    row += 1

In [ ]:

tfidf_sent_vectors_cv = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance_cv): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors_cv.append(sent_vec)
    row += 1

In [ ]:

tfidf_sent_vectors_test = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance_test): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors_test.append(sent_vec)
    row += 1

In [ ]:
# Please write all the code with proper documentation

X_train_tfw2v, X_cv_w2v, X_test_tfw2v = tfidf_sent_vectors_train, tfidf_sent_vectors_cv, tfidf_sent_vectors_test

auc_train=[]
ks_train=[]
auc_cv=[]
ks_cv=[]

for i in range(1,50,4):
    knn = KNeighborsClassifier(n_neighbors=i, algorithm='kd_tree') #try distance cosine too
    knn.fit(X_train_tfw2v, y_train_w2v)
    # y_pred = knn.predict(X_test_tfw2v)
    proba = knn.predict_proba(X_train_tfw2v)[:,1]
    auc_train.append(roc_auc_score(y_train_w2v, proba))
    ks_train.append(i)
    #############################################################
    proba = knn.predict_proba(X_cv_w2v)[:,1]
    auc_cv.append(roc_auc_score(y_cv_w2v, proba))
    ks_cv.append(i)


In [ ]:
######## AUC PLOTTING ########
ax=sns.lineplot(ks_train,auc_train)
ax=sns.lineplot(ks_cv,auc_cv)
ax.set(xlabel='Differnt K values',ylabel='roc_auc_score')
ax.set_title('AUC Score vs Different K value Curve')
plt.show()

######## best_k ##############A
best_k = ks_cv[auc_cv.index(max(auc_cv))]
print("Best K for KNN Model from cv_data is: ",best_k, "for cooresponding AUC value of:",max(auc_cv))


In [ ]:


######### FRESH-MODEL FOR BEST_K ######################
knn = KNeighborsClassifier(n_neighbors=best_k,algorithm='kd_tree')
knn.fit(X_train_tfw2v, y_train_w2v)
y_pred = knn.predict(X_test_tfw2v)
proba = knn.predict_proba(X_test_tfw2v)[:,1]


fpr, tpr, threshold = roc_curve(y_test_w2v, proba)
roc_auc = auc(fpr, tpr)
print("AUC score on test_data with best_k: ",roc_auc)
#############################################
proba = knn.predict_proba(X_train_tfw2v)[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(y_train_w2v, proba)
roc_auc_train = auc(fpr_train, tpr_train)

k_8 = best_k
set_8_auc = roc_auc
############# ROC ######################
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'g', label = 'AUC_train = %0.3f' % roc_auc_train)
plt.plot(fpr, tpr, 'b', label = 'AUC_test = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()


In [ ]:


y_true = y_test_w2v
y_pred = y_pred

from sklearn.metrics import confusion_matrix

array = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(array, index = [i for i in "01"],
                  columns = [i for i in "01"])


plt.figure(figsize = (5,5))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, linewidths=6.3, linecolor='pink', cbar=False, square=True)
plt.show()

# [6] Conclusions

In [ ]:
# Please compare all your models using Prettytable library
# Reference: http://zetcode.com/python/prettytable/

from prettytable import PrettyTable
x= PrettyTable()

x.field_names = ["Vectoriser", "Model", "Hyper Parameter", "AUC"]

x.add_row(["BOW", "Brute", k_1, set_1_auc])
x.add_row(["TFIDF", "Brute", k_2, set_2_auc])
x.add_row(["AVG-W2V", "Brute", k_3, set_3_auc])
x.add_row(["TFIDF W2V", "Brute", k_4, set_4_auc])
x.add_row(["----", "----", "----", "----"])
x.add_row(["BOW", "KD-Tree", k_5, set_5_auc])
x.add_row(["TFIDF", "KD-Tree", k_6, set_6_auc])
x.add_row(["AVG-W2V", "KD-Tree", k_7, set_7_auc])
x.add_row(["TFIDF W2V", "KD-Tree", k_8, set_8_auc])

print(x)